In [4]:
import json
import nltk
from nltk.tokenize import sent_tokenize

# Scarica i dati necessari per il tokenizzatore (se non già presenti)
nltk.download('punkt')

def process_json_file(input_file, output_file):
    # Carica il file JSON di input
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Lista per accumulare le informazioni a livello di frase
    output_sentences = []

    # Itera per ogni documento (identificato dal suo pmid)
    for paper_id, entry in data.items():
        metadata = entry.get("metadata", {})
        title = metadata.get("title", "")
        abstract = metadata.get("abstract", "")
        
        # Combina title e abstract; se preferisci lavorare separatamente, puoi adattare la logica
        combined_text = title + " " + abstract
        
        # Effettua la tokenizzazione in frasi
        sentences = sent_tokenize(combined_text)
        
        # Estrai la lista delle entità (i text_span generati dal NER)
        ner_entities = entry.get("entities", [])
        # Costruiamo una lista di entità (stringhe) da cercare nelle frasi
        entity_texts = [ent.get("text_span", "") for ent in ner_entities if "text_span" in ent]
        
        # Per ogni frase, controlla quali entità sono presenti
        for idx, sentence in enumerate(sentences):
            found_entities = []
            for ent_text in entity_texts:
                # Se l'entità non è vuota e compare nella frase
                if ent_text and ent_text in sentence:
                    found_entities.append(ent_text)
            # Rimuove eventuali duplicati
            found_entities = list(set(found_entities))
            
            # Salva le informazioni della frase nell'output
            output_sentences.append({
                "pmid": paper_id,
                "sent_id": idx,
                "sentence": sentence,
                "entities": found_entities  # lista di entità trovate in questa frase
            })
    
    # Scrive l'output in un file JSON
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_sentences, f, indent=2, ensure_ascii=False)
    
    print(f"Le frasi processate sono state salvate in {output_file}")

# Imposta i percorsi dei file di input e output
input_file = 'train_platinum.json'
output_file = 'tokenized_sentences_with_entities.json'

process_json_file(input_file, output_file)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Le frasi processate sono state salvate in tokenized_sentences_with_entities.json
